In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')

In [ ]:
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk_stopwords = stopwords.words('english')

wordnet_lemmatizer = WordNetLemmatizer()


In [ ]:
def lemSentence(msg):
    tokens = word_tokenize(msg)
    lem=[]
    for i in tokens:
        lem.append(wordnet_lemmatizer.lemmatize(i,pos="v"))
        lem.append(" ")
    return "".join(lem)
def clean(msg):
    msg = msg.translate(str.maketrans('','',string.punctuation))
    msg = msg.translate(str.maketrans('','',string.digits))
    msg = [word for word in word_tokenize(msg) if not word.lower() in nltk_stopwords]
    msg = ' '.join(msg)
    msg = lemSentence(msg)
    return msg

In [ ]:
df['clean_text'] = df.question_text.apply(lambda x: clean(x))
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df['clean_text'],df['target'],test_size=0.2,random_state=42)

In [ ]:
from sklearn.metrics import precision_score, recall_score,f1_score, accuracy_score

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
model = make_pipeline(TfidfVectorizer(), MultinomialNB(alpha=1))
model.fit(x_train,y_train)
pred = model.predict(x_test)
print('f1_score',f1_score(y_test, pred))
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, pred)))
model1 = make_pipeline(TfidfVectorizer(), SGDClassifier(random_state=0))
model1.fit(x_train,y_train)
pred1 = model1.predict(x_test)
print('f1_score',f1_score(y_test, pred1))
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, pred1)))

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
model = make_pipeline(CountVectorizer(), MultinomialNB(alpha=1))
model.fit(x_train,y_train)
pred = model.predict(x_test)
print('f1_score',f1_score(y_test, pred))
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, pred)))
model1 = make_pipeline(CountVectorizer(), SGDClassifier(random_state=0))
model1.fit(x_train,y_train)
pred1 = model1.predict(x_test)
print('f1_score',f1_score(y_test, pred1))
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, pred1)))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
#model3 = make_pipeline(CountVectorizer(), KNeighborsClassifier(n_neighbors=3))
#model3.fit(x_train, y_train)
#pred3 = model3.predict(x_test)
#print('f1_score',f1_score(y_test, pred3))
#print('Accuracy: {:.2f}'.format(accuracy_score(y_test, pred3)))

In [ ]:
x_train.head()

In [ ]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
count_vectorizer = CountVectorizer()
model4 = RandomForestClassifier(random_state=0, max_depth=2)
vectorize_model_pipeline = Pipeline([
    ('count_vectorizer', count_vectorizer),
    ('model', model4)])
vectorize_model_pipeline.fit(x_train, y_train)
#clf = make_pipeline(CountVectorizer(), tree.DecisionTreeClassifier(random_state=0, max_depth=2))
#clf.fit(x_train, y_train)
#tree.plot_tree(clf) 
pred4 = vectorize_model_pipeline.predict(x_test)
print('f1_score',f1_score(y_test, pred4))
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, pred4)))

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mx = confusion_matrix(y_test, pred1)
conf_mx

In [ ]:
print('recall',recall_score(y_test, pred1))
print('precission',precision_score(y_test, pred1))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, pred1).ravel()
print("True Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)
print("True Positives: ",tp)


In [ ]:
precission = tp/(tp+fp)
recall=tp/(tp+fn)
print(recall)
print(precission)


In [ ]:
tp, fn, fp, tn = confusion_matrix(y_test, pred1).ravel()
print("True Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)
print("True Positives: ",tp)

In [ ]:
precission = tp/(tp+fp)
recall=tp/(tp+fn)
print(recall)
print(precission)

In [ ]:
import matplotlib.pyplot as plt
import itertools    
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix'
                          ):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', vmin = 0, 
                             vmax = 40000)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, pred1)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['0','1'],
                      title='Confusion matrix, without normalization')

In [ ]:
conf_mx1 = confusion_matrix(y_test, pred, normalize='true')
print(conf_mx1)
tn, fp, fn, tp = conf_mx1.ravel()
print("True Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)
print("True Positives: ",tp)


In [ ]:
import matplotlib.pyplot as plt
def plot_confusion_matrix(matrix):
    """If you prefer color and a colorbar"""
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    cax = ax.matshow(matrix, interpolation='nearest')
    plt.imshow(matrix, vmin = 0, 
                             vmax = 40000)
    fig.colorbar(cax)
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

In [ ]:
plot_confusion_matrix(conf_mx)

In [ ]:
plot_confusion_matrix(conf_mx1)

In [ ]:
plt.matshow(conf_mx)
plt.ylabel("True label")
plt.xlabel("Predicted label")
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score,f1_score
precision_score(y_test, pred)